# Домашняя работа №2

Используем PEP-8 проверку:

In [2]:
! pip install pycodestyle flake8 pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [ ]:
Импортируем нужные модули:

In [11]:
! pip install pymystem3
! pip install pymorphy2
from pymystem3 import Mystem
m = Mystem()
from pprint import pprint
import json
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import collections

# №2

Замеряем время работы, анализируем текст, сохраняем в формате json:

In [5]:
%%time
with open('silly_frenchman.txt', encoding='utf-8') as file:
    text = file.read()
    ana = m.analyze(text) #анализируем текст с помощью mystem
with open('mystem.json', 'w', encoding='utf-8') as file: 
    json.dump(ana, file, ensure_ascii = False) #сохраняем в json

Wall time: 4min 9s


# №3

Замеряем время работы, токенизируем, парсим текст, достаем лемму и часть речи из первого разбора для каждого слова, сохраняем в формате json:

In [9]:
%%time
from pymorphy2 import MorphAnalyzer
ana_mas = []
tok = word_tokenize(text) #токенизируем текст с помощью nltk
tok_filtered = [w.lower() for w in tok if w.isalpha()] #приводим слова к нижнему регистру
for tok in tok_filtered: 
    ana = morph.parse(tok) #парсим с помощью pymorphy
    ana_mas.append(ana) 
lem_pos_mas = []
for i in ana_mas:
    first = i[0] #берем первый разбор слова
    lem_pos = [first.normal_form, first.tag.POS] #берем из разбора лемму, часть речи
    lem_pos_mas.append(lem_pos)
with open('pymorphy.json', 'w', encoding='utf-8') as file: 
    json.dump(lem_pos_mas, file, ensure_ascii = False) #cохраняем в json

Wall time: 1.2 s


# №4

Преобразуем файл из предыдущего пункта в формат питона, создадим массив, где будут лежать части речи, счетчик с частями речи и их кол-вом, посчитаем число слов и долю каждой части речи от общего числа слов:

In [22]:
lem_pos_mas = []
for line in open('pymorphy.json', encoding='utf-8'): #преобразуем в питоновский формат
    lem_pos_mas.extend(json.loads(line))
pos_mas = [] #тут части речи
for i in lem_pos_mas:
    pos = i[1] 
    pos_mas.append(pos) #добавляем части речи в массив
words_num = len(lem_pos_mas) #считаем число слов
pos_counter = Counter(pos_mas) #считаем число частей речи
pos_count_dict = dict(pos_counter) #преобразуем в словарь ("часть речи" - "кол-во")
for key in pos_count_dict.keys():
    #fraction = pos_count_dict[key]/words_num
    pos_count_dict[key] = pos_count_dict[key]/words_num #считаем, какую долю слов составляет каждая часть речи
print(pos_count_dict)

{'NOUN': 0.24434389140271492, 'PREP': 0.08257918552036199, 'VERB': 0.15610859728506787, 'ADJF': 0.08257918552036199, 'INFN': 0.03619909502262444, 'NPRO': 0.08031674208144797, 'CONJ': 0.11877828054298642, 'PRED': 0.006787330316742082, 'ADVB': 0.05090497737556561, 'PRCL': 0.07805429864253394, 'PRTF': 0.0022624434389140274, 'INTJ': 0.007918552036199095, 'GRND': 0.019230769230769232, 'NUMR': 0.012443438914027148, 'ADJS': 0.014705882352941176, 'COMP': 0.0022624434389140274, None: 0.004524886877828055}


## Топ-20 глаголов и наречий

Добавляем глаголы в счетчик, выводим 20 самых популярных:

In [23]:
verb_count = collections.Counter() #подключим счетчик глаголов
for i in lem_pos_mas:
    if i[1] == 'VERB':
        verb_count[i[0]] += 1 #если пометка глагола, добавляем в счетчик на первое место глагол
        
top_verb = verb_count.most_common(20)   #ищем 20 самых популярных глаголов
for i in top_verb:
    print(i[0], i[1])

мочь 11
есть 9
быть 6
дать 5
подать 4
подумать 4
съесть 4
приказать 3
подавать 2
ужаснуться 2
хотеть 2
послушать 2
удивиться 2
сидеть 2
стать 2
делать 2
знать 2
думать 2
иметь 2
поглядеть 2


Аналогично для наречий:

In [24]:
adv_count = collections.Counter()
for i in lem_pos_mas:
    if i[1] == 'ADVB':
        adv_count[i[0]] += 1
        
top_adv = adv_count.most_common(20)      
for i in top_adv:
    print(i[0], i[1])

много 9
здесь 3
странно 2
уже 2
уж 2
зачем 2
ужасно 2
слишком 1
сытно 1
пока 1
сразу 1
едва 1
неестественно 1
сначала 1
безнаказанно 1
шёпотом 1
этак 1
сейчас 1
тут 1
нарочно 1


# №5

Оставляем только леммы из массива с леммами и частью речи и записываем их в отдельный массив, ищем би- и триграммы, считаем их кол-во, оставляем 25 самых популярных:

In [25]:
lemms = []
for i in lem_pos_mas:
    lemms.append(i[0]) #вытаскиваем только леммы, добавляем их в массив
bi = nltk.bigrams(lemms) #генерируем биграммы
tri = nltk.trigrams(lemms) #генерируем триграммы
bi_count = collections.Counter(bi).most_common(25) #вводим счетчик биграмм, оставляем 25 самых частотных
tri_count = collections.Counter(tri).most_common(25) #вводим счетчик триграмм, оставляем 25 самых частотных

Выводим би- и триграммы:

In [26]:
for i in bi_count:
    print(i[0])

('много', 'есть')
('не', 'мочь')
('так', 'много')
('благообразный', 'господин')
('стол', 'и')
('у', 'вы')
('гора', 'блин')
('и', 'неужели')
('мочь', 'быть')
('я', 'не')
('с', 'пашот')
('пожалуй', 'дать')
('соседний', 'стол')
('подумать', 'француз')
('мочь', 'съесть')
('он', 'к')
('ещё', 'порция')
('дядя', 'франсуа')
('два', 'тарелка')
('неужели', 'он')
('дать', 'ещё')
('бог', 'мой')
('он', 'не')
('хотеть', 'умереть')
('вы', 'так')


In [27]:
for i in tri_count:
    print(i[0])

('соседний', 'стол', 'и')
('и', 'неужели', 'он')
('не', 'мочь', 'быть')
('он', 'не', 'мочь')
('так', 'много', 'есть')
('вы', 'так', 'много')
('сидеть', 'здесь', 'и')
('за', 'стол', 'сидеть')
('клоун', 'из', 'цирк')
('из', 'цирк', 'брат')
('цирк', 'брат', 'гинц')
('брат', 'гинц', 'генри')
('гинц', 'генри', 'пуркуа')
('генри', 'пуркуа', 'зайти')
('пуркуа', 'зайти', 'в')
('зайти', 'в', 'московский')
('в', 'московский', 'трактир')
('московский', 'трактир', 'тестово')
('трактир', 'тестово', 'позавтракать')
('тестово', 'позавтракать', 'дать')
('позавтракать', 'дать', 'я')
('дать', 'я', 'консоме')
('я', 'консоме', 'приказать')
('консоме', 'приказать', 'он')
('приказать', 'он', 'половый')
